# Init

In [2]:
import gym
from gym import wrappers

import numpy as np
from scipy.spatial import KDTree

# Env Settings

In [4]:
_OBS_DIM = 4
_ACT_DIM = 1

# Minimum Viable Components

In [5]:
N   = 0 # ---------------------- Number of exemplars
F   = np.zeros( (N,_OBS_DIM) ) # Approximating function
V   = np.zeros( (N,) ) # ------- Action values
A   = np.zeros( (N,_ACT_DIM) ) # Actions (Output)
KDT = None # ------------------- Spatial tree
eps = 1.0 # -------------------- Exploration probability
rad = 0.125 # ------------------ Minimum distance between exemplars (Overwrite radius)